## Visualizing single cell examples for pooled data
- Inputs are:
  - REQUIRED:
    -  **Input_gene_or_guide** - (str), it can be a string of gene symbol or guide (20 chars)
        - e.g., 'KRT28' or 'GTTGAAGAGCTAGATCAACG'
    -  **batch_name** - (str) batch name
        - e.g., '20200805_A549_WG_Screen' 
    - **rootDir** - image, metadata and overlays address is following the structure of dirs in the s3 bucket so we need to fix the root directory and everything else is being formed accroding to the s3 bucket structure 
        - e.g., '/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/'
  - OPTIONAL:
    -  **n_cells** - (int), number of single cells to be visualized, it is ignored if cell_selection_method is geometric_median
    -  **box_size** - (int), size of box in pixels for cropping a cell    
    -  **channels** - (list), list of channels you want to plot
        - e.g., ['DNA','Mito','Phalloidin','WGA','ER','Outline'] 
    -  **cell_selection_method** - (str) can be one of the following methods
        - random - generate n randomly selected cells
        - representative - clusters the data and sample from the "closest to mean cluster"
        - geometric_median - plots single sample than is the geometric median of samples
        
        
###  Run time:

- For random cells (n_cells=6) -> ~ 3-4 mins
- For representative cells (For )  -> ~ 3-4 mins
- For geometric median (for calculate the gm from 1500 random subset of total cells) -> ~ 3 mins
  

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(color_codes=True)
from sklearn import preprocessing
import matplotlib.pyplot as plt
from singlecell.read import read_single_cell_sql
from singlecell.process import extract_single_cell_samples
from singlecell.visualize import visualize_n_SingleCell
from singlecell.preprocess.filter_out_edge_single_cells import edgeCellFilter
import pooled_cell_painting_single_cell_visualization
import time
import gc

## Visualize single cells for the input gene of guide

In [ ]:
%%time

input_gene_or_guide='KRT28'
# input_gene_or_guide='GTTGAAGAGCTAGATCAACG'

# ########################## set directories
## CP186
# batch='20200805_A549_WG_Screen';

## CP257
batch='20210422_6W_CP257';


########################## set directories
rootDir='/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/'

batch_multi_name_dict={'20210422_6W_CP257':'CP257-HeLa-WG',\
                       '20200805_A549_WG_Screen':'CP186-A549-WG'}
batchName2=batch_multi_name_dict[batch]
sc_files_dir=rootDir+'workspace/software/'+batchName2+'/data/1.profiles/'+batch+'/single_cell/\
single_cell_by_guide/'
im_size=5500 # hardcoded for now, TODO: create a dictionary if this number is different for 257 vs 186

#################### options for parameters to set
n_cells=6
box_size=100

# how we select cells which can be 'random','representative','geometric_median'
cell_selection_method='geometric_median'

channels=['DNA','Mito','Phalloidin','WGA','ER','Outline']

##################### read metadata and create the file name for input guide or gene
metadata_dir=rootDir+'workspace/metadata/'+batch+'/'
metadata_orig= pd.read_csv(metadata_dir+'Barcodes.csv')

input_matched_rows = metadata_orig[(metadata_orig['gene_symbol']==input_gene_or_guide) | (metadata_orig['sgRNA']==input_gene_or_guide)]
gene=input_matched_rows.gene_symbol.unique().tolist()[0]
guides_ls=input_matched_rows.sgRNA.unique().tolist()

all_guides_gms_ls=[]
# gene_guids_ls=glob.glob(sc_files_dir+'*_'+input_gene+'.csv.gz')
gene_guids_ls=[sc_files_dir+batch+'_single_cell_normalized_ALLBATCHES__'+gi+'_'+gene+'.csv.gz' for gi in guides_ls]

for gi in gene_guids_ls:
    df_p_s=pd.read_csv(gi);

    for ch in channels:
        df_p_s["PathName_Corr"+ch]=rootDir+batch+'/images_corrected_cropped/'+df_p_s["Metadata_Foci_plate"]+'_'+df_p_s["Metadata_Foci_well"]+'/Corr'+ch
        df_p_s["FileName_Corr"+ch]="Corr"+ch+"_"+"Site_"+df_p_s["Metadata_Foci_site_location"].astype(str)+".tiff"

    df_p_s["Path_Outlines"]=rootDir+'workspace/analysis/'+batch+'/'+df_p_s["Metadata_Foci_plate"]+'-'+df_p_s["Metadata_Foci_well"]+'-'+df_p_s["Metadata_Foci_site_location"].astype(str)+'/'\
    +'/CorrDNA_Site_'+df_p_s["Metadata_Foci_site_location"].astype(str)+'_Overlay.png'

    df_p_s["Nuclei_Location_Center_X"]=df_p_s["Cells_AreaShape_Center_X"];
    df_p_s["Nuclei_Location_Center_Y"]=df_p_s["Cells_AreaShape_Center_Y"];

    df_p_s=edgeCellFilter2(df_p_s,im_size,box_size/2);

    if df_p_s.shape[0]>0:
        df_samples,cp_features_analysis = extract_single_cell_samples(df_p_s.sample(n = np.min([1500,df_p_s.shape[0]]),\
                        replace = False).reset_index(drop=True),n_cells,cell_selection_method);
        all_guides_gms_ls.append(df_samples)

df_p_s_all=pd.concat(all_guides_gms_ls,ignore_index=True).drop_duplicates(ignore_index=True)    

if len(gene_guids_ls)>1 and df_p_s_all.shape[0]>n_cells:
    df_samples=df_p_s_all.sample(n = np.min([n_cells,df_p_s_all.shape[0]]),replace = False).reset_index(drop=True)
    
else:
    df_samples=df_p_s_all.copy()

################  visualize cells

f=visualize_n_SingleCell_pooled(channels,df_samples,box_size,im_size,title=input_gene_or_guide+'_'+cell_selection_method);

In [ ]:
# df_samples
# f=visualize_n_SingleCell_pooled(channels,df_samples,box_size,im_size,title=input_gene_or_guide+'_'+cell_selection_method);

In [ ]:
np.__version__

In [ ]:
# visualize_n_SingleCell_pooled(channels,df_samples,box_size,im_size,title=cell_selection_method);
# df_samples
# df_p_s

In [ ]:
df_p_s[df_p_s.columns[df_p_s.columns.str.contains('Alig')]]

In [ ]:
def pooled_cell_painting_single_cell_visualization(input_gene_or_guide,batch,rootDir,\
            n_cells=6,box_size=100,channels=['DNA','Mito','Phalloidin','WGA','ER','Outline'],\
                                                   cell_selection_method='random'):
    

    ########################## set directories
#     batch='20200805_A549_WG_Screen';
#     rootDir='/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/'
    sc_files_dir=rootDir+'workspace/software/CP186-A549-WG/data/1.profiles/20200805_A549_WG_Screen/single_cell/single_cell_by_guide/'

    im_size=5500 # hardcoded for now, TODO: create a dictionary if this number is different for 257

    ##################### read metadata and create the file name for input guide or gene
    metadata_dir=rootDir+'workspace/metadata/'+batch+'/'
    metadata_orig= pd.read_csv(metadata_dir+'Barcodes.csv')

    input_matched_rows = metadata_orig[(metadata_orig['gene_symbol']==input_gene_or_guide) | (metadata_orig['sgRNA']==input_gene_or_guide)]
    gene=input_matched_rows.gene_symbol.unique().tolist()[0]
    guides_ls=input_matched_rows.sgRNA.unique().tolist()

    all_guides_gms_ls=[]
    # gene_guids_ls=glob.glob(sc_files_dir+'*_'+input_gene+'.csv.gz')
    gene_guids_ls=[sc_files_dir+batch+'_single_cell_normalized_ALLBATCHES__'+gi+'_'+gene+'.csv.gz' for gi in guides_ls]

    for gi in gene_guids_ls:
        df_p_s=pd.read_csv(gi);

        for ch in channels:
            df_p_s["PathName_Corr"+ch]=rootDir+batch+'/images_corrected_cropped/'+df_p_s["Metadata_Foci_plate"]+'_'+df_p_s["Metadata_Foci_well"]+'/Corr'+ch
            df_p_s["FileName_Corr"+ch]="Corr"+ch+"_"+"Site_"+df_p_s["Metadata_Foci_site_location"].astype(str)+".tiff"

        df_p_s["Path_Outlines"]=rootDir+'workspace/analysis/'+batch+'/'+df_p_s["Metadata_Foci_plate"]+'-'+df_p_s["Metadata_Foci_well"]+'-'+df_p_s["Metadata_Foci_site_location"].astype(str)+'/'\
        +'/CorrDNA_Site_'+df_p_s["Metadata_Foci_site_location"].astype(str)+'_Overlay.png'

        df_p_s["Nuclei_Location_Center_X"]=df_p_s["Cells_AreaShape_Center_X"];
        df_p_s["Nuclei_Location_Center_Y"]=df_p_s["Cells_AreaShape_Center_Y"];

        df_p_s=edgeCellFilter2(df_p_s,im_size,box_size/2);

        if df_p_s.shape[0]>0:
            df_samples,cp_features_analysis = extract_single_cell_samples(df_p_s.sample(n = np.min([1500,df_p_s.shape[0]]),\
                            replace = False).reset_index(drop=True),n_cells,cell_selection_method);
            all_guides_gms_ls.append(df_samples)

    df_p_s_all=pd.concat(all_guides_gms_ls,ignore_index=True).drop_duplicates(ignore_index=True)    

    if len(gene_guids_ls)>1 and df_p_s_all.shape[0]>n_cells:
        df_samples=df_p_s_all.sample(n = np.min([n_cells,df_p_s_all.shape[0]]),replace = False).reset_index(drop=True)

    else:
        df_samples=df_p_s_all.copy()

    ################  visualize cells

    return visualize_n_SingleCell_pooled(channels,df_samples,box_size,im_size,title=input_gene_or_guide+'_'+cell_selection_method);


input_gene_or_guide='KRT28'
# input_gene_or_guide='GTTGAAGAGCTAGATCAACG'

batch='20200805_A549_WG_Screen';
rootDir='/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/'


pooled_cell_painting_single_cell_visualization(input_gene_or_guide,batch,rootDir);

In [ ]:
# metadaDataFiles
# metadata_orig
# df_p_s.columns
# df_p_s[df_p_s.eq(5500).any(1)]

In [ ]:
# os.listdir("/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/workspace/software/CP257-HeLa-WG/data/1.profiles/20210422_6W_CP257/single_cell/")

In [ ]:
# python restore_intelligent.py imaging-platform "projects/2015_10_05_DrugRepurposing_AravindSubramanian_GolubLab_Broad/workspace/backend/2016_04_01_a549_48hr_batch1/SQ00015195/"

### Read single cell data per guide takes ~1 mins

In [ ]:
%%time
file_name='/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/workspace/\
software/CP186-A549-WG/data/1.profiles/20200805_A549_WG_Screen/single_cell/\
single_cell_by_guide/20200805_A549_WG_Screen_single_cell_normalized_ALLBATCHES__GTTGAAGAGCTAGATCAACG_KRT28.csv.gz'
df_p_s0=pd.read_csv(file_name);
df_p_s0.head()

In [ ]:
# df_p_s0[df_p_s0.columns[df_p_s0.columns.str.contains('utline')]]
# df_p_s0.columns[df_p_s0.columns.str.contains('uid')]

In [ ]:
# df_p_s0['Cells_AreaShape_Center_X'].describe()

In [ ]:
# visualize_n_SingleCell_pooled(channels,df_samples,box_size,im_size,title=cell_selection_method);

### Unarchive needed data

In [ ]:
# def unarchive_file(f2unarchive,bucket_name,user_profile):
#     """
#     unarchive input file

#     Inputs:
#     bucket_name: eg, pooled-cell-painting, imaging-plateform
#     user_profile: profile name that is used for accessing the above bucket (check in sudo vim ~/.aws/credentials)
#     proj_path: path to the project eg,2018_11_20_Periscope_Calico/workspace/analysis/

#     output:
#     command to run
#     """

#     cmd='aws s3api  restore-object  --profile '+user_profile+\
#             ' --bucket '+bucket_name+'  --key projects/2018_11_20_Periscope_Calico/workspace/analysis/'+f2unarchive+'   --restore-request GlacierJobParameters={"Tier"="Standard"}'
#     return cmd
import os
user_profile='calico'
bucket_name='pooled-cell-painting'
rootDir='/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/'
# for p in plates:
#     for w in wells:
#         for s in sites:
# batch='20200805_A549_WG_Screen';

batch='20210422_6W_CP257'
plate_well_dirs_ls=os.listdir(rootDir+'/workspace/analysis/'+batch)
for di in plate_well_dirs_ls:
    f2unarchive='workspace/analysis/'+batch+'/'+di+'/CorrDNA_Site_'+di.split('-')[-1]+'_Overlay.png'
    if os.path.exists(rootDir+f2unarchive):
        cmd='aws s3api  restore-object  --profile '+user_profile+\
        ' --bucket '+bucket_name+'  --key projects/2018_11_20_Periscope_Calico/'+\
        f2unarchive+'   --restore-request GlacierJobParameters={"Tier"="Standard"}'
        print(cmd)
        os.system(cmd)

In [ ]:
# unarchive image folders
plate_well_ls=os.listdir('/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/20210422_6W_CP257/images_corrected_cropped')
# python restore_intelligent.py pooled-cell-painting "projects/2018_11_20_Periscope_Calico/20200805_A549_WG_Screen/images_corrected_cropped/"
channels=['DNA','Mito','Phalloidin','WGA','ER']

for pli in plate_well_ls:
    for chi in channels:
        os.system('python ~/imaging-backup-scripts/restore_intelligent.py pooled-cell-painting "projects/2018_11_20_Periscope_Calico/20210422_6W_CP257/images_corrected_cropped/'+\
pli+'/Corr'+chi+'"')

In [ ]:
plate_well_dirs_ls[0].split('-')[-1]

In [ ]:
# plate_well_ls

## Plot geometric median of all guides of an input gene

In [ ]:
%%capture
import glob
rootDir='/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/'

# batch='20200805_A549_WG_Screen';
batch='20210422_6W_CP257';


batch_multi_name_dict={'20210422_6W_CP257':'CP257-HeLa-WG',\
                   '20200805_A549_WG_Screen':'CP186-A549-WG'}
batchName2=batch_multi_name_dict[batch]
sc_files_dir=rootDir+'workspace/software/'+batchName2+'/data/1.profiles/'+batch+'/single_cell/single_cell_by_guide/'
im_size=5500 # hardcoded for now, TODO: create a dictionary if this number is different for 257 vs 186

### metadata 
metadata_dir=rootDir+'workspace/metadata/'+batch+'/'
metadata_orig= pd.read_csv(metadata_dir+'Barcodes.csv')

# input_gene='KRT28'
n_cells=10
box_size=100

# how we select cells which can be 'random','representative','geometric_median'
cell_selection_method='geometric_median'
channels=['DNA','Mito','Phalloidin','WGA','ER','Outline']

genes_ls=metadata_orig.gene_symbol.unique().tolist()

for igi in range(9248,len(genes_ls)):
    input_gene=genes_ls[igi]
    all_guides_gms_ls=[]
    gene_guids_ls=glob.glob(sc_files_dir+'*_'+input_gene+'.csv.gz')
    for gi in gene_guids_ls:
        df_p_s=pd.read_csv(gi);
        
        for ch in channels:
            df_p_s["PathName_Corr"+ch]=rootDir+batch+'/images_corrected_cropped/'+df_p_s["Metadata_Foci_plate"]+'_'+df_p_s["Metadata_Foci_well"]+'/Corr'+ch
            df_p_s["FileName_Corr"+ch]="Corr"+ch+"_"+"Site_"+df_p_s["Metadata_Foci_site_location"].astype(str)+".tiff"

        df_p_s["Path_Outlines"]=rootDir+'workspace/analysis/'+batch+'/'+df_p_s["Metadata_Foci_plate"]+'-'+df_p_s["Metadata_Foci_well"]+'-'+df_p_s["Metadata_Foci_site_location"].astype(str)+'/'\
        +'/CorrDNA_Site_'+df_p_s["Metadata_Foci_site_location"].astype(str)+'_Overlay.png'

        df_p_s["Nuclei_Location_Center_X"]=df_p_s["Cells_AreaShape_Center_X"];
        df_p_s["Nuclei_Location_Center_Y"]=df_p_s["Cells_AreaShape_Center_Y"];
        
        df_p_s,_=edgeCellFilter(df_p_s,image_width=im_size,edge_margin=box_size/2);

        if df_p_s.shape[0]>0:
            df_samples,cp_features_analysis = extract_single_cell_samples.extract_single_cell_samples(df_p_s.sample(n = np.min([1000,df_p_s.shape[0]]),\
                            replace = False).reset_index(drop=True),n_cells,cell_selection_method);
            all_guides_gms_ls.append(df_samples)

    all_guides_gms_df=pd.concat(all_guides_gms_ls,ignore_index=True).drop_duplicates(ignore_index=True)
    
    if all_guides_gms_df.shape[0]==1:
        all_guides_gms_df=pd.concat([all_guides_gms_df]*2, ignore_index=True)    
    
    fig=visualize_n_SingleCell.visualize_n_SingleCell_pooled(channels,all_guides_gms_df,box_size,im_size,title=input_gene+'_'+cell_selection_method);
    
#     sgsfs
    
    resultsDir='/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/workspace/visualizations/'+batch+'/geometric_median_guide_level/'
    fig.savefig(resultsDir+input_gene+'.png')  
    plt.ioff()
    del fig

In [ ]:
igi

In [ ]:
igi, 5323 error

In [ ]:
# genes_ls.index('LOC653602')
# pd.concat([all_guides_gms_df]*2, ignore_index=True)
len(genes_ls)

In [ ]:
ls /home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/workspace/

In [ ]:
igi,input_gene,all_guides_gms_df.shape,gene_guids_ls

In [ ]:
os.system('mkdir -p /home/ubuntu/bucket/projects/2018_11_20_Periscope_Calico/workspace/visualizations/'+batch+'/geometric_median_guide_level')

In [ ]:
ls /home/ubuntu/bucket/projects/2018_11_20_Periscope_Calico/workspace/visualizations

In [ ]:
# metadata_orig.gene_symbol.unique().tolist()

In [ ]:
import glob
rootDir='/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/'

# batch='20200805_A549_WG_Screen';
batch='20210422_6W_CP257';


batch_multi_name_dict={'20210422_6W_CP257':'CP257-HeLa-WG',\
                   '20200805_A549_WG_Screen':'CP186-A549-WG'}
batchName2=batch_multi_name_dict[batch]
sc_files_dir=rootDir+'workspace/software/'+batchName2+'/data/1.profiles/'+batch+'/single_cell/single_cell_by_guide/'
im_size=5500 # hardcoded for now, TODO: create a dictionary if this number is different for 257 vs 186

### metadata 
metadata_dir=rootDir+'workspace/metadata/'+batch+'/'
metadata_orig= pd.read_csv(metadata_dir+'Barcodes.csv')

# input_gene='KRT28'
n_cells=10
box_size=100

# how we select cells which can be 'random','representative','geometric_median'
cell_selection_method='representative'
channels=['DNA','Mito','Phalloidin','WGA','ER','Outline']

resultsDir='/home/ubuntu/bucket/projects/2018_11_20_Periscope_Calico/workspace/visualizations/'+batch+'/'+cell_selection_method+'_guide_level/'


# genes_ls=metadata_orig.gene_symbol.unique().tolist()

genes_ls=['ATP6V1A', 'ATP6V1B1', 'ATP6V1B2', 'ATP6V1C1', 'ATP6V1C2', 'ATP6V1D', 'ATP6V1E1', 'ATP6V1E2',\
'ATP6V1G1', 'ATP6V1G2', 'ATP6V1G3', 'ATP6V1F', 'ATP6V1H','ALDOB', 'CCDC115', 'VMA21', 'TMEM199',\
'TM9SF4','ALG10','DPM1','DPM2','DPM3','ALG3','ALG9','ALG12','ALG6','ALG8','ALG10B','DDOST',\
'RPN1','RPN2','STT3A','DAD1','TUSC3', 'MOGS','GANAB','MAN1A1','MAN1A2','MAN1B1','MAN1C1','STT3B',\
'MAN2C1','ENGASE','FDPS','GGPS1','NEU2','NEU4','ALG5','DHDDS','DOLPP1','DOLK','DPAGT1',\
'ALG1','ALG2','ALG11','ALG13','ALG14','ACTR2','ACTR3','ARPC2','ARPC3','ATP6AP1',\
'ATP6V0D1','ATP6V1DALG1L','TMEM251',"TOMM20","nontargeting"]


genes_ls=['ARPC3','ATP6AP1',\
'ATP6V0D1','ATP6V1DALG1L','TMEM251']

# genes_ls=['ALG12','TUSC3','FDPS','ALG5','DPAGT1','ALG1']

# # genes_ls=['TMEM251']

# Meraj_only_g_ls=['ACTR2','ACTR3','ARPC2','ARPC3','ARPC4','ATP6AP1','ATP6V0D1','ATP6V1DALG1L']

# genes_ls=['ACTR2','ACTR3','ARPC2','ARPC3','ARPC4','ATP6AP1','ATP6V0D1','ATP6V1DALG1L']
# genes_ls=['TOMM20']
# genes_ls=["nontargeting"]

for igi in range(len(genes_ls)):
    input_gene=genes_ls[igi]
#     all_guides_gms_ls=[]
    gene_guids_ls=glob.glob(sc_files_dir+'*_'+input_gene+'.csv.gz')
    for gi in gene_guids_ls[78:]:
        df_p_s=pd.read_csv(gi);
        
        for ch in channels:
            df_p_s["PathName_Corr"+ch]=rootDir+batch+'/images_corrected_cropped/'+df_p_s["Metadata_Foci_plate"]+'_'+df_p_s["Metadata_Foci_well"]+'/Corr'+ch
            df_p_s["FileName_Corr"+ch]="Corr"+ch+"_"+"Site_"+df_p_s["Metadata_Foci_site_location"].astype(str)+".tiff"

        df_p_s["Path_Outlines"]=rootDir+'workspace/analysis/'+batch+'/'+df_p_s["Metadata_Foci_plate"]+'-'+df_p_s["Metadata_Foci_well"]+'-'+df_p_s["Metadata_Foci_site_location"].astype(str)+'/'\
        +'/CorrDNA_Site_'+df_p_s["Metadata_Foci_site_location"].astype(str)+'_Overlay.png'

        df_p_s["Nuclei_Location_Center_X"]=df_p_s["Cells_AreaShape_Center_X"];
        df_p_s["Nuclei_Location_Center_Y"]=df_p_s["Cells_AreaShape_Center_Y"];
        
        df_p_s=edgeCellFilter2(df_p_s,im_size,box_size/2);
        
        #.sample(n = np.min([1500,df_p_s.shape[0]]),\
        if df_p_s.shape[0]>0:
            
            df_samples,cp_features_analysis = extract_single_cell_samples(df_p_s,n_cells,cell_selection_method);
            
            resdir=resultsDir+input_gene#+'/'+gi.split('_')[-2]
            os.system("mkdir -p "+resdir)

    #             all_guides_gms_ls.append(df_samples)

    #         all_guides_gms_df=pd.concat(all_guides_gms_ls,ignore_index=True).drop_duplicates(ignore_index=True)

    #         if all_guides_gms_df.shape[0]==1:
    #             all_guides_gms_df=pd.concat([all_guides_gms_df]*2, ignore_index=True)    

            df_samples['label']=df_p_s["Metadata_Foci_well"]+'-'+df_p_s["Metadata_Foci_site_location"].astype(str)
            fig=visualize_n_SingleCell_pooled(channels,df_samples,box_size,im_size,title=input_gene+'_'+cell_selection_method);
            gc.collect()
            
        #     resultsDir='/home/ubuntu/bucket/projects/2018_11_20_Periscope_Calico/workspace/visualizations/'+batch+'/geometric_median_guide_level/'
            fig.savefig(resdir+'/'+gi.split('_')[-2]+'.png',dpi=500)  
            plt.ioff()

In [ ]:
# ARPC2
# OverflowError: cannot convert float infinity to integer
# df_samples,cp_features_analysis = extract_single_cell_samples(df_p_s,n_cells,cell_selection_method);
# len(gene_guids_ls),igi
gene_guids_ls.index(gi)

In [ ]:
df_p_s.shape

In [ ]:
# df_p_s=df_p_s.interpolate()
df_samples,cp_features_analysis = extract_single_cell_samples(df_p_s,n_cells,cell_selection_method);

In [ ]:
ValueError: Input contains NaN, infinity or a value too large for dtype('float64').
'ALG12','TUSC3','FDPS','ALG5','DPAGT1','ALG1'

In [ ]:
OverflowError: cannot convert float infinity to integer
'ARPC3'

In [ ]:
igi,input_gene

In [ ]:
# fig=visualize_n_SingleCell_pooled(channels,df_samples,box_size,im_size,title=input_gene+'_'+cell_selection_method);

imm=skimage.io.imread("/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/20210422_6W_CP257/images_corrected_cropped/CP257L_Well3/CorrDNA/CorrDNA_Site_38.tiff")

print(imm.min(),imm.max())

In [ ]:
# plt.figure()
# plt.hist(imm,bins=1000)

image = np.array([25,28, 51, 102, 153,190, 200], dtype=np.uint8)
exposure.rescale_intensity(image,in_range=(28,190))
# array([  0, 127, 255], dtype=uint8)

In [ ]:
# igi,input_gene
# df_p_s.shape
# df_p_s

from skimage import exposure
high_contrast = exposure.rescale_intensity(imm, in_range=(imm.min(), imm.max()))

In [ ]:
plt.figure()
plt.hist(imm,bins=1000)

In [ ]:
high_contrast.min(),high_contrast.max()

In [ ]:
# skdemo.imshow_with_histogram
# import skdemo

# skdemo.imshow_with_histogram(image);

In [ ]:
igi,input_gene

In [ ]:
df_p_s[cp_features_analysis].values

In [ ]:
igi

In [ ]:
genes_ls=['ATP6V1A', 'ATP6V1B1', 'ATP6V1B2', 'ATP6V1C1', 'ATP6V1C2', 'ATP6V1D', 'ATP6V1E1', 'ATP6V1E2',\
'ATP6V1G1', 'ATP6V1G2', 'ATP6V1G3', 'ATP6V1F', 'ATP6V1H','ALDOB', 'CCDC115', 'VMA21', 'TMEM199',\
'TM9SF4','ALG10','DPM1','DPM2','DPM3','ALG3','ALG9','ALG12','ALG6','ALG8','ALG10B','DDOST',\
'RPN1','RPN2','STT3A','DAD1','TUSC3', 'MOGS','GANAB','MAN1A1','MAN1A2','MAN1B1','MAN1C1','STT3B',\
'MAN2C1','ENGASE','FDPS','GGPS1','NEU2','NEU4','ALG5','DHDDS','DOLPP1','DOLK','DPAGT1',\
          'ALG1','ALG2','ALG11','ALG13','ALG14']

In [ ]:
Meraj_g_ls=["ATP6V0D1","ATP6V1F","ATP6AP1","ATP6V1C1","ATP6V1H","ATP6V1G1","ATP6V1D"
"ALG1L","ALG3","ALG9","ALG12","DPM1","DPM2","ARPC2","ARPC3","ARPC4","ACTR2","ACTR3"]

In [ ]:
set(genes_ls) & set(Meraj_g_ls)

## Visualize channel specific representative cells

- Clustering based on channel specific features for selecting single cells

In [ ]:
import glob
rootDir='/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/'

# batch='20200805_A549_WG_Screen';
batch='20210422_6W_CP257';


batch_multi_name_dict={'20210422_6W_CP257':'CP257-HeLa-WG',\
                   '20200805_A549_WG_Screen':'CP186-A549-WG'}
batchName2=batch_multi_name_dict[batch]
sc_files_dir=rootDir+'workspace/software/'+batchName2+'/data/1.profiles/'+batch+'/single_cell/single_cell_by_guide/'
im_size=5500 # hardcoded for now, TODO: create a dictionary if this number is different for 257 vs 186

### metadata 
metadata_dir=rootDir+'workspace/metadata/'+batch+'/'
metadata_orig= pd.read_csv(metadata_dir+'Barcodes.csv')

# input_gene='KRT28'
n_cells=20
box_size=100

# how we select cells which can be 'random','representative','geometric_median'
cell_selection_method='representative'
channels=['DNA','Mito','Phalloidin','WGA','ER']

resultsDir='/home/ubuntu/bucket/projects/2018_11_20_Periscope_Calico/workspace/visualizations/'+batch+'/'+\
cell_selection_method+'_guide_level_channel_level/'

genes_ls=['ATP6V1A', 'ATP6V1B1', 'ATP6V1B2', 'ATP6V1C1', 'ATP6V1C2', 'ATP6V1D', 'ATP6V1E1', 'ATP6V1E2',\
'ATP6V1G1', 'ATP6V1G2', 'ATP6V1G3', 'ATP6V1F', 'ATP6V1H','ALDOB', 'CCDC115', 'VMA21', 'TMEM199',\
'TM9SF4','ALG10','DPM1','DPM2','DPM3','ALG3','ALG9','ALG12','ALG6','ALG8','ALG10B','DDOST',\
'RPN1','RPN2','STT3A','DAD1','TUSC3', 'MOGS','GANAB','MAN1A1','MAN1A2','MAN1B1','MAN1C1','STT3B',\
'MAN2C1','ENGASE','FDPS','GGPS1','NEU2','NEU4','ALG5','DHDDS','DOLPP1','DOLK','DPAGT1',\
'ALG1','ALG2','ALG11','ALG13','ALG14','ACTR2','ACTR3','ARPC2','ARPC3','ATP6AP1',\
'ATP6V0D1','ATP6V1DALG1L','TMEM251',"TOMM20","nontargeting"]

# genes_ls=metadata_orig.gene_symbol.unique().tolist()

genes_ls=["nontargeting"]

# Meraj_only_g_ls=['ACTR2','ACTR3','ARPC2','ARPC3','ARPC4','ATP6AP1','ATP6V0D1','ATP6V1DALG1L']

# genes_ls=['ACTR2','ACTR3','ARPC2','ARPC3','ARPC4','ATP6AP1','ATP6V0D1','ATP6V1DALG1L']
# genes_ls=['TOMM20']
# genes_ls=["nontargeting"]

for igi in range(len(genes_ls)):
    input_gene=genes_ls[igi]
    all_guides_gms_ls=[]
    gene_guids_ls=glob.glob(sc_files_dir+'*_'+input_gene+'.csv.gz')
    for gi in gene_guids_ls[77:]:
        df_p_s=pd.read_csv(gi);
        
        for ch in channels:
            df_p_s["PathName_Corr"+ch]=rootDir+batch+'/images_corrected_cropped/'+df_p_s["Metadata_Foci_plate"]+'_'+df_p_s["Metadata_Foci_well"]+'/Corr'+ch
            df_p_s["FileName_Corr"+ch]="Corr"+ch+"_"+"Site_"+df_p_s["Metadata_Foci_site_location"].astype(str)+".tiff"

        df_p_s["Path_Outlines"]=rootDir+'workspace/analysis/'+batch+'/'+df_p_s["Metadata_Foci_plate"]+'-'+df_p_s["Metadata_Foci_well"]+'-'+df_p_s["Metadata_Foci_site_location"].astype(str)+'/'\
        +'/CorrDNA_Site_'+df_p_s["Metadata_Foci_site_location"].astype(str)+'_Overlay.png'

        df_p_s["Nuclei_Location_Center_X"]=df_p_s["Cells_AreaShape_Center_X"];
        df_p_s["Nuclei_Location_Center_Y"]=df_p_s["Cells_AreaShape_Center_Y"];
        
        df_p_s=edgeCellFilter2(df_p_s,im_size,box_size/2);
        
        #.sample(n = np.min([1500,df_p_s.shape[0]]),\
#         channel_name_in_feats=['DAPI_Painting','ConA','Mito','Phalloidin','WGA']
        align_column_ch_name_map={'DNA':'DAPI_Painting','ER':'ConA','Mito':'Mito','Phalloidin':'Phalloidin','WGA':'WGA'}
        
        if df_p_s.shape[0]>0:
            for ch in channels:
                channel_feats=df_p_s.columns[df_p_s.columns.str.contains('_'+align_column_ch_name_map[ch]+'|_X|_Y|FileName_|Path|Metadata_')]
                df_samples,cp_features_analysis = extract_single_cell_samples(df_p_s[channel_feats],n_cells,cell_selection_method);

                resdir=resultsDir+input_gene+'/'+gi.split('_')[-2]
                os.system("mkdir -p "+resdir)

        #             all_guides_gms_ls.append(df_samples)

        #         all_guides_gms_df=pd.concat(all_guides_gms_ls,ignore_index=True).drop_duplicates(ignore_index=True)

        #         if all_guides_gms_df.shape[0]==1:
        #             all_guides_gms_df=pd.concat([all_guides_gms_df]*2, ignore_index=True)    

                df_samples['label']=df_p_s["Metadata_Foci_well"]+'-'+df_p_s["Metadata_Foci_site_location"].astype(str)
#                 fig=visualize_n_SingleCell_pooled(channels,df_samples,box_size,im_size,title=input_gene+'_'+cell_selection_method);
                channels2=[ch,'Outline']
                fig=visualize_n_SingleCell_pooled(channels2,df_samples,box_size,im_size,title=input_gene+'_'+cell_selection_method);
                gc.collect()
            #     resultsDir='/home/ubuntu/bucket/projects/2018_11_20_Periscope_Calico/workspace/visualizations/'+batch+'/geometric_median_guide_level/'
                fig.savefig(resdir+'/'+ch+'.png',dpi=500)  
                plt.ioff()

In [ ]:
# df_samples.shape
# np.power(2,16)
# channels
gene_guids_ls.index(gi)

In [ ]:
df_samples.columns[df_samples.columns.str.contains('FileName')]

In [ ]:
df_p_s.columns.str.contains('_ConA')

In [ ]:
# ls /home/ubuntu/bucket/projects/2018_11_20_Periscope_Calico/workspace/visualizations/20210422_6W_CP257/representative_guide_level_channel_level/

In [ ]:
2500*454

### For each guide save one cell from each plate well site 
- There are 454 non-targeting guids and 9 plates and 6 wells and ~100 sites
- For each guide select 1 randomly selected cell from each plate well site -> "Metadata_site"  metadata column 
  - Which is about 2500 cells per guide -> this got to memory error
  
- Instead i sample 10 cells from each plat well -> results are saved in 
https://imaging-platform.s3.us-east-1.amazonaws.com/projects/2018_11_20_Periscope_Calico/workspace/visualizations/20210422_6W_CP257/nontargeting_dfs

- Total shape would be about

In [ ]:
import glob
batch='20210422_6W_CP257';
rootDir='/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/'
batch_multi_name_dict={'20210422_6W_CP257':'CP257-HeLa-WG',\
                   '20200805_A549_WG_Screen':'CP186-A549-WG'}
batchName2=batch_multi_name_dict[batch]
sc_files_dir=rootDir+'workspace/software/'+batchName2+'/data/1.profiles/'+batch+'/single_cell/single_cell_by_guide/'
im_size=5500 

# resultsDir='/home/ubuntu/bucket/projects/2018_11_20_Periscope_Calico/workspace/visualizations/'+batch+'/'+\
# cell_selection_method+'_guide_level_channel_level/'


input_gene="nontargeting"
all_guides_gms_ls=[]
gene_guids_ls=glob.glob(sc_files_dir+'*_'+input_gene+'.csv.gz')
nontarget_df_ls=[]
ix=0
for gi in gene_guids_ls:
    df_p_s=pd.read_csv(gi);
    ix+=1
    print(ix)
#     afdsdf
#     nontarget_df_ls.append(df_p_s.groupby('Metadata_site').sample(n = 1, replace = False).reset_index(drop=True)) # this gives memory error
    nontarget_df_ls.append(df_p_s.groupby(['Metadata_Foci_plate','Metadata_Foci_well']).sample(n = 10, replace = True).reset_index(drop=True)) # this gives memory error    
#     for ch in channels:
#         df_p_s["PathName_Corr"+ch]=rootDir+batch+'/images_corrected_cropped/'+df_p_s["Metadata_Foci_plate"]+'_'+df_p_s["Metadata_Foci_well"]+'/Corr'+ch
#         df_p_s["FileName_Corr"+ch]="Corr"+ch+"_"+"Site_"+df_p_s["Metadata_Foci_site_location"].astype(str)+".tiff"

#     df_p_s["Path_Outlines"]=rootDir+'workspace/analysis/'+batch+'/'+df_p_s["Metadata_Foci_plate"]+'-'+df_p_s["Metadata_Foci_well"]+'-'+df_p_s["Metadata_Foci_site_location"].astype(str)+'/'\
#     +'/CorrDNA_Site_'+df_p_s["Metadata_Foci_site_location"].astype(str)+'_Overlay.png'

#     df_p_s["Nuclei_Location_Center_X"]=df_p_s["Cells_AreaShape_Center_X"];
#     df_p_s["Nuclei_Location_Center_Y"]=df_p_s["Cells_AreaShape_Center_Y"];

#     df_p_s=edgeCellFilter2(df_p_s,im_size,box_size/2);

#     #.sample(n = np.min([1500,df_p_s.shape[0]]),\
#     if df_p_s.shape[0]>0:

#         df_samples,cp_features_analysis = extract_single_cell_samples(df_p_s,n_cells,cell_selection_method);
            
#         nontarget_df_ls.append(df_samples)

In [ ]:
for gi in gene_guids_ls[251:]:
    df_p_s=pd.read_csv(gi);
    ix+=1
    print(ix)
#     afdsdf
#     nontarget_df_ls.append(df_p_s.groupby('Metadata_site').sample(n = 1, replace = False).reset_index(drop=True)) # this gives memory error
    nontarget_df_ls.append(df_p_s.groupby(['Metadata_Foci_plate','Metadata_Foci_well']).sample(n = 10, replace = True).reset_index(drop=True)) # this gives memory error    


In [ ]:
df_p_s=pd.read_csv(gene_guids_ls[2]);
df_p_s.groupby('Metadata_site').sample(n = 1, replace = False).reset_index(drop=True).shape  


In [ ]:
# 540*545
https://imaging-platform.s3.us-east-1.amazonaws.com/projects/2018_11_20_Periscope_Calico/workspace/visualizations/20210422_6W_CP257/nontargeting_dfs

In [ ]:
rootDir+'/workspace/visualizations/'+batch+'/nontargeting_dfs/'
git rm --cached <filename>

In [ ]:
pd.concat(nontarget_df_ls,axis=0,ignore_index=True).to_csv('/home/ubuntu/bucket/projects/2018_11_20_Periscope_Calico//workspace/visualizations/'+batch+'/nontargeting_dfs/guide_ten_per_plate_well.csv',index=False)


In [ ]:
# df_p_s.groupby('Metadata_site').sample(n = 1, replace = False).reset_index(drop=True).shape
l

In [ ]:
ls /home/ubuntu/bucket/projects/2018_11_20_Periscope_Calico/workspace/visualizations/

In [ ]:
df_p_s.groupby(['Metadata_Foci_plate','Metadata_Foci_well']).sample(n = 10, replace = False).reset_index(drop=True)

In [ ]:
df_p_s.groupby(['Metadata_Foci_plate','Metadata_Foci_well']).size()

In [ ]:
# df_p_s.groupby(['Metadata_Foci_plate','Metadata_Foci_site']).size()
df_p_s.groupby(['Metadata_Foci_plate','Metadata_Foci_well','Metadata_site']).size()#.groupby(['Metadata_Foci_plate'])

In [ ]:
df_p_s.columns[df_p_s.columns.str.contains('site')]

In [ ]:
files = glob.glob(path + "/**/*.txt", recursive = True)